In [12]:
import pandas as pd

In [13]:
dataset=pd.read_csv("/home/moeenuddin/Desktop/all_projects_ML/Remote_Employee_Burnout_Predictor/Remote_Employee_Burnout_Predictor/dataset/raw/wfh_burnout_dataset.csv")

In [14]:
df=dataset

In [15]:
df.head()

,user_id,day_type,work_hours,screen_time_hours,meetings_count,breaks_taken,after_hours_work,app_switches,sleep_hours,task_completion,isolation_index,fatigue_score,burnout_score,burnout_risk
0,129,Weekday,7.74,5.99,3,7,0,50,7.50,74.80,4,3.51,17.23,Low
1,24,Weekday,8.48,7.23,3,5,0,63,7.31,86.36,4,5.15,24.01,Low
2,147,Weekend,8.39,7.14,1,5,0,27,6.73,89.23,8,7.85,51.46,Medium
3,42,Weekday,12.48,12.23,4,1,1,89,6.55,64.47,5,10.00,55.50,Medium
4,99,Weekend,8.85,7.35,4,6,0,40,8.03,98.62,3,3.45,11.95,Low


In [16]:
df["day_type"].value_counts()

day_type
Weekday    1435
Weekend     565
Name: count, dtype: int64

In [17]:
df.columns

Index(['user_id', 'day_type', 'work_hours', 'screen_time_hours',
       'meetings_count', 'breaks_taken', 'after_hours_work', 'app_switches',
       'sleep_hours', 'task_completion', 'isolation_index', 'fatigue_score',
       'burnout_score', 'burnout_risk'],
      dtype='str')

In [18]:
df[df["user_id"]==129].shape[0]

16

In [19]:
df["user_id"].value_counts()

user_id
81     25
47     23
96     22
26     22
21     21
       ..
3       8
35      7
66      7
116     7
61      7
Name: count, Length: 150, dtype: int64

In [24]:
import pandas as pd
import os

# 1. Define your specific paths
# Input: Your original raw file
input_file = '/home/moeenuddin/Desktop/all_projects_ML/Remote_Employee_Burnout_Predictor/Remote_Employee_Burnout_Predictor/dataset/raw/wfh_burnout_dataset.csv'

# Output: Your new file in the same raw folder
output_file = '/home/moeenuddin/Desktop/all_projects_ML/Remote_Employee_Burnout_Predictor/Remote_Employee_Burnout_Predictor/dataset/raw/new_columns_dataset.csv'

# 2. Load the dataset
if os.path.exists(input_file):
    df = pd.read_csv(input_file)
    print(f"✅ Successfully loaded dataset from: {input_file}")
else:
    print(f"❌ Error: Could not find the file at {input_file}")
    exit()

# 3. Prepare the data
# We sort by user_id to ensure chronological order for the rolling memory
df = df.sort_values(by=['user_id']).reset_index(drop=True)

print("⏳ Engineering new columns (Adding Memory to the data)...")

# ---------------------------------------------------------
# NEW COLUMN 1: 7-Day Rolling Average of Work Hours
# ---------------------------------------------------------
df['work_hours_7d_avg'] = df.groupby('user_id')['work_hours'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
).round(2)

# ---------------------------------------------------------
# NEW COLUMN 2: 3-Day Accumulated Fatigue
# ---------------------------------------------------------
df['fatigue_3d_sum'] = df.groupby('user_id')['fatigue_score'].transform(
    lambda x: x.rolling(window=3, min_periods=1).sum()
).round(2)

# ---------------------------------------------------------
# NEW COLUMN 3: Is it a Weekend? (Context Flag)
# ---------------------------------------------------------
df['is_weekend'] = df['day_type'] == 'Weekend'

# ---------------------------------------------------------
# NEW COLUMN 4: 7-Day Rolling Average of Meetings
# ---------------------------------------------------------
df['meetings_7d_avg'] = df.groupby('user_id')['meetings_count'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
).round(1)

# 4. Save to your requested location
df.to_csv(output_file, index=False)

print(f"🚀 Done! New dataset saved as: {output_file}")

# 5. Preview the results to confirm
print("\n--- Preview of the Engineered Columns ---")
print(df[['user_id', 'day_type', 'work_hours', 'work_hours_7d_avg', 'fatigue_3d_sum']].head(10))

✅ Successfully loaded dataset from: /home/moeenuddin/Desktop/all_projects_ML/Remote_Employee_Burnout_Predictor/Remote_Employee_Burnout_Predictor/dataset/raw/wfh_burnout_dataset.csv
⏳ Engineering new columns (Adding Memory to the data)...
🚀 Done! New dataset saved as: /home/moeenuddin/Desktop/all_projects_ML/Remote_Employee_Burnout_Predictor/Remote_Employee_Burnout_Predictor/dataset/raw/new_columns_dataset.csv

--- Preview of the Engineered Columns ---
   user_id day_type  work_hours  work_hours_7d_avg  fatigue_3d_sum
0        1  Weekday        7.34               7.34            2.73
1        1  Weekday        7.32               7.33            8.12
2        1  Weekday        7.28               7.31           11.88
3        1  Weekday        8.26               7.55           14.75
4        1  Weekday        7.23               7.49           13.06
5        1  Weekday        7.47               7.48           11.58
6        1  Weekend        8.23               7.59           10.06
7       